# Bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('pnad_0.csv', encoding='unicode_escape', delimiter=';')
df.head()

/var/folders/2l/7pnfw34j5615gbx_v66_1vw40000gn/T/ipykernel_3585/2351567434.py:1: DtypeWarning: Columns (35,101,102,145,146,148,149,153,155,156,158,159) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pnad_0.csv', encoding='unicode_escape', delimiter=';')


,Unnamed: 0,Ano,Trimestre,UF,Capital,RM_RIDE,UPA,Estrato,Númerodeselecaododomicílio,Painel,...,Rendim.efetivoqqtrabalho,Pqñññtrab,Pqñññtrab_A,Hrshabituaisemtodostrab,Hrsefetivasnotrab.princ_A,Hrsefetivasnotrab.secun_A,Hrsefetivasnosoutrostrab_A,Hrsefetivasemtodostrab,Faixahrshabituaistrab.princ,Faixahrsefetivastrab.princ
0,1,2018,4,Pernambuco,NaN,NaN,260026030,2653013,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2018,4,Santa Catarina,NaN,Região Metropolitana de Florianópolis (SC),420078589,4220011,4,7,...,2500.0,NaN,NaN,44.0,NaN,NaN,NaN,44.0,40 a 44 horas,40 a 44 horas
2,3,2018,4,Acre,NaN,NaN,120003892,1250010,11,7,...,200.0,NaN,NaN,40.0,NaN,NaN,NaN,0.0,40 a 44 horas,Até 14 horas
3,4,2018,4,Rio Grande do Sul,NaN,NaN,430000389,4351021,7,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2018,4,Pará,NaN,NaN,150024364,1551021,11,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.shape

(22451, 217)

In [4]:
df.dropna(subset=["Rendim.efetivoqqtrabalho"], inplace=True) #Removendo as linhas com a coluna Rendim.efetivoqqtrabalho (target)

In [5]:
df.shape # Verificando o novo shape

(9162, 217)

In [6]:
# Vendo as variáveis com a correlação igual ou superior a 80% em relação a target
corr = df.corr()
features = []
for i in range(0, 207):
    if (corr["Rendim.efetivoqqtrabalho"].sort_values(ascending=False)[i] >= 0.8): # Se quiser pegar correlações menores, alterar o valor ao lado.
        features.append(corr["Rendim.efetivoqqtrabalho"].sort_values(ascending=False).keys()[i]) # Adicionando o nome da feature no array
    else:
        break

features # Checando as features selecionadas

['Rendim.efetivoqqtrabalho',
 'Rendim.efetivotrab.princ',
 'Valordorend.efe.emdinheiro',
 'Rendim.habitualqqtrabalho',
 'Rendim.habitualtrab.princ',
 'Valordorend.hab.emdinheiro']

In [7]:
df_model = df[features] # adicionando as features selecionadas em um novo dataframe

In [8]:
df_model.info() # Checando o tipo de dado das features selecionadas e se tem valores nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9162 entries, 1 to 22445
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rendim.efetivoqqtrabalho    9162 non-null   float64
 1   Rendim.efetivotrab.princ    9161 non-null   float64
 2   Valordorend.efe.emdinheiro  9137 non-null   float64
 3   Rendim.habitualqqtrabalho   9162 non-null   float64
 4   Rendim.habitualtrab.princ   9161 non-null   float64
 5   Valordorend.hab.emdinheiro  9137 non-null   float64
dtypes: float64(6)
memory usage: 501.0 KB


In [9]:
df_model.fillna(method="ffill",inplace=True) # Preenchendo os valores das colunas que estão vazios / nulos

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna - Referência técnica

/var/folders/2l/7pnfw34j5615gbx_v66_1vw40000gn/T/ipykernel_3585/3215514947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model.fillna(method="ffill",inplace=True) # Preenchendo os valores das colunas que estão vazios / nulos


In [10]:
df_model.info() # Checando como ficaram as variáveis após o preenchimento

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9162 entries, 1 to 22445
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rendim.efetivoqqtrabalho    9162 non-null   float64
 1   Rendim.efetivotrab.princ    9162 non-null   float64
 2   Valordorend.efe.emdinheiro  9162 non-null   float64
 3   Rendim.habitualqqtrabalho   9162 non-null   float64
 4   Rendim.habitualtrab.princ   9162 non-null   float64
 5   Valordorend.hab.emdinheiro  9162 non-null   float64
dtypes: float64(6)
memory usage: 501.0 KB


In [11]:
# Criando variáveis independentes e dependente
X = df_model.drop("Rendim.efetivoqqtrabalho", axis=1)
y = df_model[["Rendim.efetivoqqtrabalho"]]

In [12]:
# Separando o modelo em treino e teste. Dataframe de teste é de 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
# Instancia da regressão linear
model = LinearRegression()

In [14]:
# Treinamento do modelo
model.fit(X_train, y_train)

LinearRegression()

In [15]:
# Avaliação do modelo
model.score(X_test, y_test)

0.9953973505846402